In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.linalg import solve
import scipy
import scipy.integrate
from scipy.integrate import solve_ivp


## Orbit solution

In [2]:
def big_system(t, Y_M, f, Jacf, dim, A, m, w, gamma):
    # Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M) 
    X_M = np.zeros((dim+dim**2),dtype=np.float64) #We solve simustanuously d+d*d ODEs to link up the solution and the monodromy matrix computation
    M = Y_M[dim:].reshape(dim, dim, order="F")  # Reshape the flat array back into a dim^2 x dim matrix
    dM_dt = Jacf(t,Y_M[:dim],gamma) @ M  # Compute the matrix derivative
    X_M[:dim] = f(t, Y_M[:dim],A,m,w,gamma)
    X_M[dim:] = dM_dt.flatten(order = "F")
    return X_M

In [ ]:
def monodromy_syst(t, Monodromy, ystar, Jacf, dim, A, a, b, gamma, eta):
    # Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M) 
    # X_M = np.zeros((dim+dim**2)) #We solve simustanuously d+d*d ODEs to link up the solution and the monodromy matrix computation
    M = Monodromy.reshape(dim, dim, order="F")  # Reshape the flat array back into a dim^2 x dim matrix
    dM_dt = Jacf(t,ystar, A, a, b, gamma, eta) @ M  # Compute the matrix derivative
    # X_M[:dim] = f(t, Y_M[:dim],A,a,b,gamma,eta)
    # X_M[dim:] = dM_dt.flatten(order = "F")
    return dM_dt.flatten(order = "F")

In [ ]:
Y_M = np.asarray([0,1,1,0,0,1])
M = np.array([[1,0], [0,1]])
A = 5
a = b = 0.05
gamma, eta = 1e-3, 0.0
dim =2
X_M = big_system(0, Y_M, f, Gradf, 2, A, a, b, gamma, eta)


print(X_M)
monodromy_t = X_M[dim:]#.reshape(dim**2, -1, order='F')
# print(M.flatten().reshape(2,2))

monodromy_t.reshape(dim, dim, order='F')

In [ ]:
#Concat the whole matrix
A11 = M
b = np.array([5,6])
c = np.array([7,8])
d = 9
top = np.hstack((A11, b.reshape(-1,1)))  # Horizontal stacking of A11 and A12 = b
bottom = np.hstack((c.reshape(1,-1),np.array([[d]])))  # Horizontal stacking of A21 =c and A22=d
Mat = np.vstack((top, bottom)) 
print(Mat)

In [3]:
def Newton_orbite_T_known(f, X_0,T,A, m, w, gamma, Jacf, Max_iter, epsilon):
    
    dim = np.shape(X_0)[0] #The problem's dimension

    #________________________________INITIALISATION_____________________________________
    k, Xstar_0 = 0, X_0
    sol = solve_ivp(fun=f,t_span=[0.0, T], y0=X_0, t_eval=[T],
                     method='DOP853', args = (A, m, w, gamma), option={"rtol": 1e-10,"atol":1e-10})
    G_Xstar_0 = sol.y[:,-1] - Xstar_0 #We take the solution at t=T
    #The complete rhd of the Newton system
    B = G_Xstar_0
    
    monodromy = np.eye(dim,dtype=np.float64) #Initialisation of the monodromy matrix

    X_by_iter , Norm_DeltaX= np.zeros((Max_iter+1,dim),dtype=np.float64),np.zeros((Max_iter+1),dtype=np.float64)
    X_by_iter[0,:] = Xstar_0
    Norm_DeltaX[0] = 1 # np.linalg.norm(B, ord=2)
    norm_delta_X =1
    #_____________Newton iteration loop____________________
    while (norm_delta_X> epsilon  and k<Max_iter):
        
        #- Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M)
        X_M = np.zeros((dim+dim**2),dtype=np.float64) 

        # Flatten the monodromy to a 1D array, as solve_ivp needs a vector
        X_M[:dim] = Xstar_0
        X_M[dim:] = monodromy.flatten(order='F')
        #  Solve the system of ODEs
        big_sol= solve_ivp(big_system, [0.0,T], X_M,
                            t_eval=[T],
                            args=(f,Jacf,dim,A, m, w, gamma),
                            method='DOP853',option={"rtol": 1e-12,"atol":1e-14}) #It's a function of t
        
        X_M = big_sol.y[:,-1] #dimension d+d*d
        Xstar_T = X_M[:dim] # The solution at T
        monodromy = X_M[dim:] #We take M(T)
        monodromy = monodromy.reshape(dim,dim, order = "F") #Back to the square matrix format
        #______Computation of DeltaX and DeltaT_____
        #Concat the whole matrix
        Mat = monodromy - np.eye(dim,dtype=np.float64)
        
        #Right hand side concatenation
        G_Xstar_0 = Xstar_T - Xstar_0
        B = G_Xstar_0

        Delta_X = solve(Mat,-B) #Contain Delta_X and Delta_T
        # Delta_X = np.linalg.lstsq(Mat, -B, rcond=None)[0]
        print("Iteration", k, "\n")
        print("Norm(Dx) = ", norm_delta_X,"\n")

        #Updating
        Xstar_0 = Delta_X + Xstar_0
        norm_delta_X = np.linalg.norm(Delta_X,ord=2)
        k = k+1
        X_by_iter[k,:] = Xstar_0
        Norm_DeltaX[k] = norm_delta_X
    # eigenvalues, eigenvectors = np.linalg.eig(monodromy)
    return k, X_by_iter, Norm_DeltaX, monodromy

### APPLICATION TO THE LASER EQUATION

In [ ]:
K = lambda a,b,X: 1 + a/(1+b*X)
def f(t,X, A, a, b, gamma, eta): 
        return np.array([(X[1] - K(a,b,X[0]))*X[0] + eta*X[1],
                                  (A-X[1]-X[1]*X[0])*gamma])

Jacf = lambda t,X,A,a,b,gamma,eta: np.array([[a*b*X[0]/(1+b*X[0])**2 + X[1] - K(a,b,X[0]), 
                                                 X[0]+ eta],
                                                 [-gamma*X[1],-gamma*(1+X[0])]])

#Definition of the rescaled system: t = t/T
def F(t,X, A, a, b, gamma, eta): 
        return X[2]*np.array([(X[1] - K(a,b,X[0]))*X[0] + eta*X[1],
                                  (A-X[1]-X[1]*X[0])*gamma,0])

# Jac_F = lambda t,X,A,a,b,gamma,eta: np.array([[X[2]*(a*b*X[0]/(1+b*X[0])**2 + X[1] - K(a,b,X[0])), 
                                                #  X[2]*(X[0]+ eta), (X[1] - K(a,b,X[0]))*X[0] + eta*X[1]]
                                                #  [-X[2]*gamma*X[1],-X[2]*gamma*(1+X[0]),(A-X[1]-X[1]*X[0])*gamma],
                                                #   [0,0,0]])

# # Variable chang L = log(I)
# fL = lambda t,X,A,a,b,gamma,eta: np.array([(X[1] - K(a,b,np.exp(X[0]))) + eta*X[1]/np.exp(X[0]),
#                                   (A-X[1]-X[1]*np.exp(X[0]))*gamma])

# GradfL = lambda t,X,A,a,b,gamma,eta: np.array([[a*b*np.exp(X[0])/(1+b*np.exp(X[0]))**2 -eta*X[1]*np.exp(-X[0]), 
#                                                  1 + eta*np.exp(-X[0])],
#                                                  [-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0]))]])

s = lambda t,X,A,gamma: gamma*(A-X[1]-X[1]*X[0])
ds_T = lambda t,X : 0.0
GradS_D = lambda t,X,a,b,gamma,eta,: np.array([-gamma*X[1],-gamma*(1+X[0])]) #Phase condition over D

# GradS_I = lambda t,X,a,b,gamma,eta: np.array([a*b*X[0]/(1+b*X[0])**2 + X[1] - K(a,b,X[0]),X[0]+ eta]) #Phase condition over I


In [ ]:
def verif_jac(f,Jac,t,X, A, a, b, gamma, eta,h):
    dim = len(X)
    Jac_num = np.zeros((dim,dim))
    for j in range(dim):
        e = h*np.eye(1,dim,j)[0] #Canonical vector basis

        Jac_num[:,j]=(f(t,X+e, A, a, b, gamma, eta) - f(t,X, A, a, b, gamma, eta))/h
        # print(Jac_num)
    res = Jac_num -Jac(t,X, A, a, b, gamma, eta)
    return np.linalg.norm(res)

In [ ]:
Max_iter = 200
epsilon = 1e-15
A,a,b = 5, 0.05, 0.05
gamma, eta = 1e-3, 0.0
args = (A,a,b,gamma,eta)
I0, D0 = 22.5,1.05 #1.25, 1.20#3.96356544, 1.0073404#24., 1.025
T0 = 127
X0 = np.array([I0,D0])

In [ ]:
Err = []
H = []
for k in range(10,1000,2): #[1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]:
    h = 1/(k**2)
    Err.append(verif_jac(f,Jacf,T0,X0, A, a,b, gamma, eta,h))
    H.append(h)

In [ ]:
# H = np.array([1e-6,1e-8,1e-10,1e-12,1e-14,1e-16])
fig = plt.figure()
H = np.array(H)
plt.loglog(H,Err,'+-', label='Err J_num - J_anal')
plt.loglog(H,0.006199*H,label = 'slope p=1')
plt.xlabel(r'$h=\frac{1}{k^2}, k = 10,...100$')
plt.legend()
plt.title("Numerical verification of the Jacobian analytical computation:\n Modulated Laser Model")
plt.grid()
# plt.savefig('./Results/jacobian_check.png')

In [ ]:
sol = solve_ivp(fun=f,t_span=[0.0, 50*T0],
                t_eval=np.linspace(0.0, 50*T0,100), 
                y0=X0, method='RK45', 
                args = args,vectorized=False,
                **{"rtol": 1e-3,"atol":1e-6}
                )
Y = sol.y[:,-1]
# sol2 = solve_ivp(fun=f,t_span=[0.0, 5*T0],
                # t_eval=np.linspace(0.0, 5*T0,100), 
                # y0=X0, method='RK45', 
                # args = args,vectorized=False,
                # **{"rtol": 1e-3,"atol":1e-6})

In [ ]:
print(Y)
print(sol2.y[:,-1])

In [ ]:
k_D, XX_D,Norm_B, M_D = Newton_orbite_T_known(f, X0,T0,
                                               args, Jacf, Max_iter, epsilon)


In [ ]:
k, Tstar, XX,Norm_B, M = Newton_orbit3(f,F,s, X0,T0,args, Jacf,GradS_D,ds_T, Max_iter, epsilon)

### Modulated Laser Model

In [ ]:
def f_modul(t,X, A, m, w, gamma): 
        return np.array([X[1] - 1 - m*np.cos(w*t) ,
                            (A - X[1]*(1 + np.exp(X[0])))*gamma])

Jacf_modul = lambda t,X,gamma: np.array([[0,1],
                                                [-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0]))]])
s_modul = lambda t,X,A,m,w,gamma : X[1] - 1 - m
GradS_modul = lambda t,X,A,m,w,gamma: np.array([0,1])

s_modul2 = lambda t,X, A,m,w,gamma: gamma*(A - X[1]*(1+np.exp(X[0])))
GradS_modul2 = lambda t,X,A,m,w,gamma: np.array([-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0]))])

ds_T_modul = lambda t,X: 0.0

In [ ]:
Max_iter = 200
epsilon = 1e-13
A = 1.1
m = 2.5e-2
gamma = 1e-3
w = 1e-2
eta = 0
params = (A,m,w,gamma)
L0, D0 = -5.878379158004416, 0.985996818660573#-np.log(A-1)+0.001, 1.005 #-5. ,1#0.9805 # m+1
X0 = np.array([L0,D0]) + 1e-4
T0 = 2*np.pi/w

In [ ]:
sol = solve_ivp(fun=f_modul,t_span=[0.0, 5*T0],
                t_eval=[T0], #np.linspace(0.0,5*T0, 1000), 
                y0=X0, method='RK45', 
                args = params,vectorized=False,
                **{"rtol": 1e-10,"atol":1e-10}
                )

In [ ]:
sol.y[:,-1]

In [ ]:
k, XX,Norm_DeltaX, M = Newton_orbite_T_known(f_modul, X0,T0, A, m, w, gamma, Jacf_modul, Max_iter, epsilon)


### Ploting

In [ ]:
# print(Tstar_D[k_I-1])
# print(XX_D[k_I-1])
# print(Tstar[k])
print(XX[k])
k

In [ ]:
fig, ax = plt.subplots(1,3,sharex='all')
Tab = np.asarray(XX[:k])
ax[0].plot(np.arange(k),Tab[:,1],'+-')
ax[0].set_xlabel("Newton iterations")
ax[0].set_ylabel(f"$D_0$")

ax[1].plot(np.arange(k),Tab[:,0],'+-')
ax[1].set_xlabel("Newton iterations")
ax[1].set_ylabel(f"$L_0$")
ax[2].semilogy(np.arange(1,k),Norm_DeltaX[1:k],'x-')
ax[2].set_xlabel("Newton iterations")
ax[2].set_ylabel(f"$\parallel \Delta X \parallel$")
fig.set_size_inches((10,8))
fig.suptitle(f'Modulated Laser: Fixed point by Newton with fixed period $T=%.3f$ \n $m = %.3f$, $L_0 = %.3f, D_0= %.3f $' % (T0, m, L0,D0))
# fig.suptitle('(Rescaled approach Laser model :\n I0 = %.3f, D0 %.3f} )' % (I0,D0))

fig.subplots_adjust(left=0.09, bottom=0.51, right=0.95, top=0.90, hspace=0.55, wspace=0.55)
plt.savefig('./Results/Modulated_Laser_T_known.png')

In [ ]:
X0 = np.array(XX[k])
print(X0)
# X0 = np.array([-2.30,  1.02])
# T = Tstar_D[k_D-1]
sol = solve_ivp(fun=f_modul,t_span=[0.0, 5*T0],
                t_eval=np.linspace(0.0,5*T0, 10000), 
                y0=X0, method='RK45', 
                args = params,vectorized=False,
                **{"rtol": 1e-12,"atol":1e-14}
                )

In [ ]:
X = sol.y
times_intervall = sol.t
# plt.plot(times_intervall,X[0,:])
# plt.xlim([0,1100])
plt.plot(times_intervall,X[0,:])
plt.grid()
plt.xlabel("time")
plt.ylabel("L0")
plt.show()
plt.plot(times_intervall,X[1,:])
plt.ylabel("D0")
plt.show()
plt.plot(X[0,:], X[1,:])
plt.grid()

X.shape


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(M_D)
print("Eigen values of the Monodromy matrix ",eigenvalues)
Tab = np.asarray(XX_I[:k_I])
plt.plot(np.arange(k_I), Tab[:,0], label = "I0")
plt.plot(np.arange(k_I),Tab[:,1], label = "D0")
plt.xlabel("Newton iteration")
plt.ylabel("I0,D0")
plt.legend()
# plt.plot(np.arange(k), Tstar, label = "T")
XX_I[k_I-1]
